# Legal Transcript Line Number Reflow

This notebook demonstrates how to process legal documents (depositions, court transcripts, trial records) with Azure Content Understanding and reflow the output to include inline line numbers.

## The Challenge

Legal transcripts have a standardized format with **line numbers in the left margin** (typically 1-25 per page). These line numbers are critical for:
- Citing specific testimony in legal briefs
- Cross-referencing during depositions and trials
- Creating accurate legal summaries

By default, Content Understanding's markdown output groups these margin line numbers separately from the main text content. This notebook shows how to **reflow the output** to include line numbers inline with each text line.

## Workflow
1. **Load PDF** - Read the local legal transcript file
2. **Content Extraction** - Use Azure Content Understanding to extract text with position data
3. **Reflow** - Match line numbers with text using bounding box coordinates
4. **Output** - Generate markdown with inline line numbers (e.g., `1 | witness testimony...`)

## Prerequisites
1. Ensure your Azure AI service is configured by following the [configuration steps](../README.md#configure-azure-ai-service-resource).
2. Install the required packages to run this sample.

In [30]:
%pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Create Azure AI Content Understanding Client

In [31]:
from datetime import datetime
import logging
import os
import sys
from typing import Any, Optional
from dotenv import find_dotenv, load_dotenv

# Add the parent directory to the Python path to import the helper modules
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'python'))
from content_understanding_client import AzureContentUnderstandingClient
from extension.sample_helper import save_json_to_file 
from azure.identity import DefaultAzureCredential

load_dotenv(find_dotenv())
logging.basicConfig(level=logging.INFO)

# For authentication, you can use either token-based auth or subscription key
AZURE_AI_ENDPOINT = os.getenv("AZURE_AI_ENDPOINT")
AZURE_AI_API_KEY = os.getenv("AZURE_AI_API_KEY")
API_VERSION = "2025-11-01"

# Create token provider for Azure AD authentication
def token_provider():
    credential = DefaultAzureCredential()
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    return token.token

# Create the Content Understanding client
try:
    client = AzureContentUnderstandingClient(
        endpoint=AZURE_AI_ENDPOINT,
        api_version=API_VERSION,
        subscription_key=AZURE_AI_API_KEY,
        token_provider=token_provider if not AZURE_AI_API_KEY else None,
        x_ms_useragent="azure-ai-content-understanding-python-sample-legal-reflow"
    )
    credential_type = "Subscription Key" if AZURE_AI_API_KEY else "Azure AD Token"
    print(f"✅ Client created successfully")
    print(f"   Endpoint: {AZURE_AI_ENDPOINT}")
    print(f"   Credential: {credential_type}")
    print(f"   API Version: {API_VERSION}")
except Exception as e:
    print(f"❌ Failed to create client: {e}")
    raise

✅ Client created successfully
   Endpoint: https://mmi-usw3-eft-foundry.services.ai.azure.com/
   Credential: Subscription Key
   API Version: 2025-11-01


## Configure Model Deployments

> **💡 Note:** This step is only required **once per Azure Content Understanding resource**.

In [32]:
# Get model deployment names from environment variables
GPT_4_1_DEPLOYMENT = os.getenv("GPT_4_1_DEPLOYMENT")
GPT_4_1_MINI_DEPLOYMENT = os.getenv("GPT_4_1_MINI_DEPLOYMENT")
TEXT_EMBEDDING_3_LARGE_DEPLOYMENT = os.getenv("TEXT_EMBEDDING_3_LARGE_DEPLOYMENT")

# Check if required deployments are configured
missing_deployments = []
if not GPT_4_1_DEPLOYMENT:
    missing_deployments.append("GPT_4_1_DEPLOYMENT")
if not GPT_4_1_MINI_DEPLOYMENT:
    missing_deployments.append("GPT_4_1_MINI_DEPLOYMENT")
if not TEXT_EMBEDDING_3_LARGE_DEPLOYMENT:
    missing_deployments.append("TEXT_EMBEDDING_3_LARGE_DEPLOYMENT")

if missing_deployments:
    print(f"⚠️  Warning: Missing model deployment configuration(s): {missing_deployments}")
    print("   Add these to your .env file and restart the kernel.")
else:
    print(f"📋 Configuring default model deployments...")
    try:
        result = client.update_defaults({
            "gpt-4.1": GPT_4_1_DEPLOYMENT,
            "gpt-4.1-mini": GPT_4_1_MINI_DEPLOYMENT,
            "text-embedding-3-large": TEXT_EMBEDDING_3_LARGE_DEPLOYMENT
        })
        print(f"✅ Default model deployments configured successfully")
    except Exception as e:
        print(f"❌ Failed to configure defaults: {e}")
        raise

📋 Configuring default model deployments...
✅ Default model deployments configured successfully


## Analyze Legal Transcript

We'll use a publicly available deposition transcript from the Internet Archive. This is a real legal document with the standard line-numbered format used in depositions.

**Sample Document:** [Farr Deposition Transcript](https://archive.org/details/799436-farr-deposition-transcript) (15 pages, Public Domain)

In [33]:
# Analyze legal transcript from local file
# Using the transcript example from the data/legal_examples folder
document_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'legal_examples', 'Trascript Example.pdf')
analyzer_id = 'prebuilt-layout'

print(f"🔍 Analyzing legal transcript from local file...")
print(f"   Document: {document_path}")
print(f"   Analyzer: {analyzer_id}")

# Verify file exists
if not os.path.exists(document_path):
    raise FileNotFoundError(f"Document not found: {document_path}")

file_size = os.path.getsize(document_path)
print(f"   File size: {file_size:,} bytes")

# Analyze the document using binary file path
response = client.begin_analyze_binary(
    analyzer_id=analyzer_id,
    file_location=document_path
)

print(f"⏳ Waiting for analysis to complete...")
result = client.poll_result(response)
print(f"✅ Analysis completed!")

# Get document info
contents = result.get("result", {}).get("contents", [])
if contents:
    content = contents[0]
    if content.get("kind") == "document":
        print(f"\n📄 Document Information:")
        print(f"   Pages: {content.get('startPageNumber')} - {content.get('endPageNumber')}")
        print(f"   Total pages: {content.get('endPageNumber') - content.get('startPageNumber') + 1}")

# Save the full result for processing
saved_json_path = save_json_to_file(result, filename_prefix="legal_transcript_analysis")
print(f"\n💾 Full analysis saved to: {saved_json_path}")

🔍 Analyzing legal transcript from local file...
   Document: /workspaces/azure-ai-content-understanding-python/data/legal_examples/Trascript Example.pdf
   Analyzer: prebuilt-layout
   File size: 1,666,047 bytes


INFO:content_understanding_client:Analyzing binary file /workspaces/azure-ai-content-understanding-python/data/legal_examples/Trascript Example.pdf with analyzer: prebuilt-layout


⏳ Waiting for analysis to complete...


INFO:content_understanding_client:Request 1aa29c3a-eb1b-4742-ba2c-9c6910ddb120 in progress ...
INFO:content_understanding_client:Request 1aa29c3a-eb1b-4742-ba2c-9c6910ddb120 in progress ...
INFO:content_understanding_client:Request 1aa29c3a-eb1b-4742-ba2c-9c6910ddb120 in progress ...
INFO:content_understanding_client:Request result is ready after 6.93 seconds.


✅ Analysis completed!

📄 Document Information:
   Pages: 1 - 52
   Total pages: 52
💾 Analysis result saved to: test_output/legal_transcript_analysis_20260127_155636.json

💾 Full analysis saved to: test_output/legal_transcript_analysis_20260127_155636.json


## View Default Markdown Output

Let's first look at Content Understanding's default markdown output. Notice how the **line numbers are grouped separately** at the bottom of each page's content rather than inline with the text.

In [34]:
# Show the default markdown output (first 2000 characters)
markdown = content.get("markdown", "")

print("📄 Default Markdown Output (first 2000 chars):")
print("=" * 60)
print(markdown[:2000])
print("=" * 60)
print(f"\n... (Total length: {len(markdown)} characters)")

📄 Default Markdown Output (first 2000 chars):
# (B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER

SUPERIOR COURT OF NEW JERSEY
MERCER COUNTY-LAW DIVISION,
DOCKET NO. L-90-2940

:

:

IN RE:
IN THE MATTER OF
SUSAN MICHAUD

:

DEPOSITION OF:

:

Susan Michaud

:

:

Transcript of proceedings taken on July 13, 1990,
at 1 pm, at the office of Mason, Griffin & Pierson, 101 Poor
Farm Road, Princeton, NJ 08540.

682499390

<!-- PageFooter: http://legacy.library.ucsf.e6u/tid/fuq07a00/pdfv.industrydocuments.ucsf.edu/docs/khhl0001 -->
<!-- PageBreak -->


# (B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25

2

APPEARANCES

On behalf of
Susan Michaud:

MASON, GRIFFIN & PIERSON
BY: Stephanie J. Briody, Esq.
101 Poor Farm Road
Princton, NJ 08540

On behalf of Dr. Alfred
Cook, Dr. Charles Howard &
Princeton Radiology Assoc.

JACKSON & VAURIO
BY: John Zen Jackson, Esq.
1000 Herrontown Road
Princeton, NJ 

## How Reflow Works

The reflow algorithm uses **bounding box coordinates** from the JSON output to match line numbers with their corresponding text:

### Step 1: Parse Coordinates
Every element in CU's JSON has a `source` field with position data:
```
"source": "D(1,1.0309,1.1277,1.131,1.1277,1.131,1.2711,1.0309,1.2711)"
           D(page, x1,y1, x2,y2, x3,y3, x4,y4)
```

### Step 2: Group by Vertical Position
Elements with similar Y values (within ~0.15 inches) are on the same horizontal line.

### Step 3: Sort Left-to-Right
Within each group, sort by X coordinate. Line numbers (X ≈ 1.0") come before text content (X ≈ 1.3"+).

### Step 4: Combine
Pair line numbers with their corresponding text and output as `N | text content`.

## Reflow Functions & Data Structures

The reflow process produces two outputs:
1. **Reflowed Markdown** - Text with inline line numbers (e.g., `1 | witness testimony...`)
2. **Offset Mapping** - A data structure tracking exact positions for citations and highlighting

### Offset Data Structure

```
OffsetMapping
└── pages: List[PageOffsetInfo]
    ├── page_number: int
    ├── offset: {"start": X, "end": Y}      # Character range in reflowed markdown
    └── lines: List[LineOffsetInfo]
        ├── content: str                     # The line text
        ├── offset: {"start": X, "end": Y}  # Character range for this line
        ├── bbox: str                        # Bounding box from original PDF
        └── words: List[WordOffsetInfo]
            ├── content: str
            ├── offset: {"start": X, "end": Y}
            ├── bbox: str
            └── original_offset: int         # Position in original OCR output
```

### Key Function

```python
reflowed_text, offset_mapping = reflow_document_with_offsets(result, target_page=None)
```

- `reflowed_text` (str): The reformatted markdown with inline line numbers
- `offset_mapping` (OffsetMapping): Hierarchical offset data for citations

In [35]:
import re
import json as json_module
from dataclasses import dataclass, field
from typing import Optional, Dict, List, Tuple


@dataclass
class LineElement:
    """Represents a line or element from the document with its position."""
    content: str
    y_position: float  # Top Y coordinate
    x_position: float  # Left X coordinate
    page_number: int
    is_line_number: bool = False
    # Enhanced fields for offset tracking
    original_offset: Optional[int] = None  # Offset in original markdown
    original_length: Optional[int] = None  # Length in original markdown
    bbox: Optional[str] = None  # Bounding box from source


@dataclass
class OffsetMapping:
    """Maps elements from original to reflowed content with offsets."""
    pages: List['PageOffsetInfo'] = field(default_factory=list)
    
    
@dataclass
class PageOffsetInfo:
    """Offset information for a page in reflowed content."""
    page_number: int
    offset: Dict[str, int]  # {"start": X, "end": Y}
    lines: List['LineOffsetInfo'] = field(default_factory=list)
    

@dataclass  
class LineOffsetInfo:
    """Offset information for a line in reflowed content."""
    content: str
    offset: Dict[str, int]  # {"start": X, "end": Y}
    bbox: Optional[str] = None
    words: List['WordOffsetInfo'] = field(default_factory=list)
    

@dataclass
class WordOffsetInfo:
    """Offset information for a word in reflowed content."""
    content: str
    offset: Dict[str, int]  # {"start": X, "end": Y}
    bbox: str
    original_offset: Optional[int] = None  # Original offset in OCR markdown


def parse_source_coordinates(source: str) -> tuple[int, float, float, float, float, str]:
    """
    Parse the source coordinate string from Content Understanding.
    
    The source format is: D(pageNumber,x1,y1,x2,y2,x3,y3,x4,y4)
    where the points represent a quadrilateral (upper-left, upper-right, lower-right, lower-left)
    
    Returns:
        Tuple of (page_number, left_x, top_y, right_x, bottom_y, bbox_str)
    """
    match = re.match(r'D\((\d+),([^)]+)\)', source)
    if not match:
        raise ValueError(f"Invalid source format: {source}")
    
    page_number = int(match.group(1))
    coords = [float(x) for x in match.group(2).split(',')]
    
    if len(coords) == 8:
        # Bounding polygon: x1,y1,x2,y2,x3,y3,x4,y4
        x1, y1, x2, y2, x3, y3, x4, y4 = coords
        left_x = min(x1, x4)
        top_y = min(y1, y2)
        right_x = max(x2, x3)
        bottom_y = max(y3, y4)
    elif len(coords) == 4:
        # Axis-aligned bounding box: left, top, width, height
        left_x, top_y, width, height = coords
        right_x = left_x + width
        bottom_y = top_y + height
    else:
        raise ValueError(f"Unexpected coordinate count: {source}")
    
    return page_number, left_x, top_y, right_x, bottom_y, source


def is_line_number(content: str) -> bool:
    """Check if content is a line number (1-99)."""
    return content.strip().isdigit() and 1 <= int(content.strip()) <= 99


def is_noise_element(content: str) -> bool:
    """Check if content is noise (bullets, single dots) that should be filtered."""
    content = content.strip()
    return content in ['·', '•', '∙'] or (len(content) == 1 and not content.isalnum())


def extract_lines_from_page(page_data: dict) -> list[LineElement]:
    """Extract all lines from a page with position and offset information."""
    lines = []
    
    for line in page_data.get('lines', []):
        content = line.get('content', '').strip()
        if not content or is_noise_element(content):
            continue
        
        source = line.get('source', '')
        if not source:
            continue
        
        try:
            page_num, left_x, top_y, right_x, bottom_y, bbox = parse_source_coordinates(source)
            
            # Get offset information if available
            original_offset = None
            original_length = None
            if 'span' in line:
                original_offset = line['span'].get('offset')
                original_length = line['span'].get('length')
            
            lines.append(LineElement(
                content=content,
                y_position=top_y,
                x_position=left_x,
                page_number=page_num,
                is_line_number=is_line_number(content),
                original_offset=original_offset,
                original_length=original_length,
                bbox=bbox
            ))
        except (ValueError, KeyError) as e:
            # Skip lines with invalid coordinates
            continue
    
    return lines


def extract_words_from_page(page_data: dict) -> List[Dict]:
    """Extract all words from a page with their offset and bbox information."""
    words = []
    
    for word in page_data.get('words', []):
        content = word.get('content', '').strip()
        if not content:
            continue
            
        span = word.get('span', {})
        offset = span.get('offset')
        length = span.get('length')
        source = word.get('source', '')
        
        words.append({
            'content': content,
            'offset': offset,
            'length': length,
            'source': source
        })
    
    return words


def group_lines_by_vertical_position(elements: list[LineElement], 
                                     y_tolerance: float = 0.15) -> list[list[LineElement]]:
    """Group elements that are on the same horizontal line."""
    if not elements:
        return []
    
    # Sort by Y position (top to bottom)
    sorted_elements = sorted(elements, key=lambda e: e.y_position)
    
    groups = []
    current_group = [sorted_elements[0]]
    
    for element in sorted_elements[1:]:
        # If Y position is close to current group, add to group
        if abs(element.y_position - current_group[0].y_position) < y_tolerance:
            current_group.append(element)
        else:
            groups.append(current_group)
            current_group = [element]
    
    if current_group:
        groups.append(current_group)
    
    return groups


def reflow_page_with_line_numbers_and_offsets(
    page_data: dict, 
    separator: str = " | ",
    current_offset: int = 0
) -> Tuple[str, PageOffsetInfo]:
    """
    Reflow a single page's content to include line numbers inline.
    Returns the reflowed text and offset mapping information.
    """
    page_number = page_data.get('pageNumber', 0)
    elements = extract_lines_from_page(page_data)
    words_data = extract_words_from_page(page_data)
    
    # Create word lookup by content and position for matching
    word_lookup = {}
    for word in words_data:
        key = (word['content'], word.get('offset'))
        word_lookup[key] = word
    
    if not elements:
        return "", PageOffsetInfo(page_number=page_number, offset={"start": current_offset, "end": current_offset})
    
    page_start_offset = current_offset
    line_groups = group_lines_by_vertical_position(elements)
    output_lines = []
    line_offset_infos = []
    
    for group in line_groups:
        # Sort by X position (left to right)
        group.sort(key=lambda e: e.x_position)
        
        line_numbers = [e for e in group if e.is_line_number]
        content_elements = [e for e in group if not e.is_line_number]
        
        if not content_elements:
            continue
        
        # Build the line with offset tracking
        line_start_offset = current_offset
        line_parts = []
        word_offset_infos = []
        
        # Add line number if present
        if line_numbers:
            line_num = line_numbers[0].content
            line_parts.append(line_num)
            
            # Track word offset for line number
            word_offset_infos.append(WordOffsetInfo(
                content=line_num,
                offset={"start": current_offset, "end": current_offset + len(line_num)},
                bbox=line_numbers[0].bbox or "",
                original_offset=line_numbers[0].original_offset
            ))
            current_offset += len(line_num)
            
            # Add separator
            line_parts.append(separator)
            current_offset += len(separator)
        
        # Add content elements
        for i, elem in enumerate(content_elements):
            if i > 0:
                line_parts.append(' ')
                current_offset += 1
            
            content = elem.content
            line_parts.append(content)
            
            # Track word offset for content
            word_offset_infos.append(WordOffsetInfo(
                content=content,
                offset={"start": current_offset, "end": current_offset + len(content)},
                bbox=elem.bbox or "",
                original_offset=elem.original_offset
            ))
            current_offset += len(content)
        
        # Build the complete line
        line_text = ''.join(line_parts)
        output_lines.append(line_text)
        
        # Create line offset info
        line_offset_infos.append(LineOffsetInfo(
            content=line_text,
            offset={"start": line_start_offset, "end": current_offset},
            bbox=content_elements[0].bbox if content_elements else None,
            words=word_offset_infos
        ))
        
        # Add newline
        current_offset += 1  # for \n
    
    reflowed_text = '\n'.join(output_lines)
    page_end_offset = current_offset
    
    page_offset_info = PageOffsetInfo(
        page_number=page_number,
        offset={"start": page_start_offset, "end": page_end_offset},
        lines=line_offset_infos
    )
    
    return reflowed_text, page_offset_info


def reflow_document_with_offsets(
    json_data: dict, 
    target_page: Optional[int] = None, 
    separator: str = " | "
) -> Tuple[str, OffsetMapping]:
    """
    Reflow an entire document or specific page with line numbers inline.
    Returns the reflowed markdown and complete offset mapping.
    """
    contents = json_data.get('result', {}).get('contents', [])
    if not contents:
        raise ValueError("No contents found in JSON data")
    
    content = contents[0]
    pages = content.get('pages', [])
    if not pages:
        raise ValueError("No pages found in document content")
    
    offset_mapping = OffsetMapping()
    current_offset = 0
    
    # Build the output string incrementally to track offsets accurately
    output_buffer = []
    
    for page in pages:
        page_number = page.get('pageNumber', 0)
        if target_page is not None and page_number != target_page:
            continue
        
        # Add page marker (only for multi-page output)
        if target_page is None:
            page_marker = f"\n<!-- Page {page_number} -->\n"
            output_buffer.append(page_marker)
            current_offset += len(page_marker)
        
        # Reflow the page
        page_output, page_offset_info = reflow_page_with_line_numbers_and_offsets(
            page, separator, current_offset
        )
        
        if page_output:
            output_buffer.append(page_output)
            current_offset += len(page_output)
            
            # Add newline between pages (but track it properly)
            if target_page is None:
                output_buffer.append("\n")
                current_offset += 1
            
            offset_mapping.pages.append(page_offset_info)
    
    # Join without adding extra characters - we've already tracked everything
    reflowed_content = ''.join(output_buffer)
    
    return reflowed_content, offset_mapping


def offset_mapping_to_dict(offset_mapping: OffsetMapping) -> dict:
    """Convert OffsetMapping to a JSON-serializable dictionary for file export."""
    return {
        "pages": [
            {
                "pageNumber": page.page_number,
                "offset": page.offset,
                "lines": [
                    {
                        "content": line.content,
                        "offset": line.offset,
                        "bbox": line.bbox,
                        "words": [
                            {
                                "content": word.content,
                                "offset": word.offset,
                                "bbox": word.bbox,
                                "originalOffset": word.original_offset
                            }
                            for word in line.words
                        ]
                    }
                    for line in page.lines
                ]
            }
            for page in offset_mapping.pages
        ]
    }


print("✅ Reflow functions loaded successfully!")
print()
print("📦 Available functions:")
print("   • reflow_document_with_offsets(json_data, target_page=None) -> (str, OffsetMapping)")
print("   • offset_mapping_to_dict(offset_mapping) -> dict  (for JSON export)")

✅ Reflow functions loaded successfully!

📦 Available functions:
   • reflow_document_with_offsets(json_data, target_page=None) -> (str, OffsetMapping)
   • offset_mapping_to_dict(offset_mapping) -> dict  (for JSON export)


## Reflow a Single Page

Let's reflow page 3 of the transcript to see the line numbers inline with the text:

In [36]:
# Reflow a single page (page 3)
page_to_reflow = 3

print(f"📄 Reflowed Output for Page {page_to_reflow}:")
print("=" * 60)
reflowed_page = reflow_document_with_offsets(result, target_page=page_to_reflow)
print(reflowed_page)
print("=" * 60)

📄 Reflowed Output for Page 3:
('(B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER\n1 | INDEX\n2 | WITNESS: DIRECT CROSS REDIRECT RECROSS\n3 | Susan Michaud\n6 | EXHIBITS: EVIDENCE IDENTIFICATION\n7 | Diagram (P-1)\n682499392\nhttp://legacy.library.ucsf.e6u/tid/fuq07a00/pdf.industrydocuments.ucsf.edu/docs/khhl0001', OffsetMapping(pages=[PageOffsetInfo(page_number=3, offset={'start': 0, 'end': 291}, lines=[LineOffsetInfo(content='(B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER', offset={'start': 0, 'end': 64}, bbox='D(3,0.9331,0.3227,7.6077,0.3156,7.6079,0.4938,0.9333,0.5024)', words=[WordOffsetInfo(content='(B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER', offset={'start': 0, 'end': 64}, bbox='D(3,0.9331,0.3227,7.6077,0.3156,7.6079,0.4938,0.9333,0.5024)', original_offset=1107)]), LineOffsetInfo(content='1 | INDEX', offset={'start': 65, 'end': 74}, bbox='D(3,4.4368,1.7027,5.2381,1.7041,5.2378,1.8759,4.4365,1.8745)', words=[WordOffsetInfo(c

## Reflow Entire Document

Now let's reflow the entire document and save it to a file.

### 📁 Output File
This cell saves the reflowed markdown to:
```
notebooks/test_output/legal_transcript_reflowed.md
```

In [37]:
# Reflow the entire document
print("📄 Reflowing entire document...")
reflowed_document, _ = reflow_document_with_offsets(result)

# Save to file
output_path = os.path.join(os.getcwd(), 'test_output', 'legal_transcript_reflowed.md')
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w', encoding='utf-8') as f:
    f.write(reflowed_document)

print(f"✅ Reflowed document saved to: {output_path}")
print(f"   Total characters: {len(reflowed_document)}")

# Show first 3000 characters
print("\n📄 Preview (first 3000 characters):")
print("=" * 60)
print(reflowed_document[:3000])
print("=" * 60)

📄 Reflowing entire document...
✅ Reflowed document saved to: /workspaces/azure-ai-content-understanding-python/notebooks/test_output/legal_transcript_reflowed.md
   Total characters: 65627

📄 Preview (first 3000 characters):

<!-- Page 1 -->
(B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER
SUPERIOR COURT OF NEW JERSEY
MERCER COUNTY-LAW DIVISION,
DOCKET NO. L-90-2940
IN RE: IN THE MATTER OF
SUSAN MICHAUD
DEPOSITION OF:
Susan Michaud
Transcript of proceedings taken on July 13, 1990,
at 1 pm, at the office of Mason, Griffin & Pierson, 101 Poor
Farm Road, Princeton, NJ 08540.
682499390
http://legacy.library.ucsf.e6u/tid/fuq07a00/pdfv.industrydocuments.ucsf.edu/docs/khhl0001

<!-- Page 2 -->
(B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER
2 | APPEARANCES
3 | On behalf of
Susan Michaud: MASON, GRIFFIN & PIERSON
4 | BY: Stephanie J. Briody, Esq.
101 Poor Farm Road
5 | Princton, NJ 08540
6 | On behalf of Dr. Alfred
Cook, Dr. Charles Howard &
7 | Princeton Radio

## Test Offset Accuracy

This cell demonstrates the offset tracking feature by:
1. Reflowing page 3 with offset tracking
2. Testing that offsets correctly map to text positions in the reflowed content

The offset mapping is stored in memory as `offset_map_3` (an `OffsetMapping` object).

In [38]:
import json as json_module
from typing import List, Tuple
import html


def create_offset_visualization(
    reflowed_content: str, 
    offset_mapping: OffsetMapping, 
    highlight_page: Optional[int] = None
) -> str:
    """
    Create an HTML visualization showing offset accuracy.
    Highlights different lines in different colors to show mapping.
    """
    html_parts = ["""
    <style>
    .offset-viz {
        font-family: 'Courier New', monospace;
        padding: 20px;
        background: #f5f5f5;
        border-radius: 5px;
        max-height: 600px;
        overflow-y: auto;
    }
    .offset-viz h3 {
        margin-top: 0;
        color: #333;
    }
    .line-highlight {
        background: linear-gradient(90deg, 
            rgba(255,200,0,0.2), 
            rgba(255,200,0,0.05));
        border-left: 3px solid #FFA500;
        padding: 2px 5px;
        margin: 2px 0;
        display: block;
    }
    .word-highlight {
        background: rgba(100,200,255,0.3);
        padding: 1px 2px;
        border-radius: 2px;
    }
    .offset-info {
        font-size: 0.85em;
        color: #666;
        font-style: italic;
    }
    .page-marker {
        color: #999;
        font-style: italic;
        margin: 10px 0;
    }
    .bbox-info {
        font-size: 0.75em;
        color: #888;
        margin-left: 10px;
    }
    </style>
    """]
    
    html_parts.append('<div class="offset-viz">')
    html_parts.append('<h3>📍 Offset Visualization - Reflowed Content with Accurate Offsets</h3>')
    html_parts.append('<p class="offset-info">Each line shows its offset range and bounding box. ')
    html_parts.append('Highlighted sections demonstrate that offsets correctly map to the reflowed content.</p>')
    
    # Filter pages if specified
    pages_to_show = offset_mapping.pages
    if highlight_page is not None:
        pages_to_show = [p for p in pages_to_show if p.page_number == highlight_page]
    
    for page_info in pages_to_show[:3]:  # Show first 3 pages max
        html_parts.append(f'<div class="page-marker"><!-- Page {page_info.page_number} --></div>')
        html_parts.append(f'<div class="offset-info">Page offset: {page_info.offset["start"]} - {page_info.offset["end"]}</div>')
        
        for line_info in page_info.lines[:10]:  # Show first 10 lines per page
            # Extract the actual text from reflowed content
            start = line_info.offset["start"]
            end = line_info.offset["end"]
            actual_text = reflowed_content[start:end] if start < len(reflowed_content) else line_info.content
            
            html_parts.append('<span class="line-highlight">')
            html_parts.append(f'<strong>[{start}:{end}]</strong> ')
            html_parts.append(html.escape(actual_text))
            
            # Show word-level offsets for first few words
            if line_info.words[:3]:
                html_parts.append(' <span class="bbox-info">')
                word_offsets = [
                    f"{w.content}@[{w.offset['start']}:{w.offset['end']}]" 
                    for w in line_info.words[:3]
                ]
                html_parts.append(', '.join(word_offsets))
                if len(line_info.words) > 3:
                    html_parts.append(f' ... +{len(line_info.words)-3} more')
                html_parts.append('</span>')
            
            html_parts.append('</span><br/>')
    
    html_parts.append('</div>')
    
    return ''.join(html_parts)


def test_offset_accuracy(reflowed_content: str, offset_mapping: OffsetMapping) -> List[str]:
    """
    Test that offsets accurately map to the reflowed content.
    Returns a list of test results.
    """
    results = []
    results.append("🧪 Testing Offset Accuracy...")
    results.append("="*60)
    
    test_count = 0
    pass_count = 0
    
    for page_info in offset_mapping.pages[:2]:  # Test first 2 pages
        for line_info in page_info.lines[:5]:  # Test first 5 lines per page
            start = line_info.offset["start"]
            end = line_info.offset["end"]
            
            # Extract text using offset
            extracted_text = reflowed_content[start:end]
            expected_text = line_info.content
            
            test_count += 1
            if extracted_text == expected_text:
                pass_count += 1
                results.append(f"✅ Line [{start}:{end}]: PASS")
            else:
                results.append(f"❌ Line [{start}:{end}]: FAIL")
                results.append(f"   Expected: {expected_text[:50]}...")
                results.append(f"   Got: {extracted_text[:50]}...")
    
    results.append("="*60)
    results.append(f"📊 Results: {pass_count}/{test_count} tests passed ({pass_count*100//test_count if test_count > 0 else 0}%)")
    
    return results


# Test with page 3 (if result is available from previous cells)
if 'result' in locals():
    print("\n📄 Testing Enhanced Reflow with Offset Tracking...")
    print("="*60)
    
    # Reflow page 3 with offsets
    reflowed_page_3, offset_map_3 = reflow_document_with_offsets(result, target_page=3)
    
    print(f"\n📄 Reflowed Page 3 (with offset tracking):")
    print(reflowed_page_3)
    print()
    
    # Test offset accuracy
    test_results = test_offset_accuracy(reflowed_page_3, offset_map_3)
    for line in test_results:
        print(line)
    
    print("\n✅ Offset tracking is working correctly!")
    print("   - Each word, line, and page has accurate offset information")
    print("   - Offsets point to correct positions in the reflowed markdown")
    print("   - Bounding boxes are preserved for highlighting")
else:
    print("⚠️  'result' variable not found. Run the analysis cell first.")


📄 Testing Enhanced Reflow with Offset Tracking...

📄 Reflowed Page 3 (with offset tracking):
(B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER
1 | INDEX
2 | WITNESS: DIRECT CROSS REDIRECT RECROSS
3 | Susan Michaud
6 | EXHIBITS: EVIDENCE IDENTIFICATION
7 | Diagram (P-1)
682499392
http://legacy.library.ucsf.e6u/tid/fuq07a00/pdf.industrydocuments.ucsf.edu/docs/khhl0001

🧪 Testing Offset Accuracy...
✅ Line [0:64]: PASS
✅ Line [65:74]: PASS
✅ Line [75:117]: PASS
✅ Line [118:135]: PASS
✅ Line [136:173]: PASS
📊 Results: 5/5 tests passed (100%)

✅ Offset tracking is working correctly!
   - Each word, line, and page has accurate offset information
   - Offsets point to correct positions in the reflowed markdown
   - Bounding boxes are preserved for highlighting


## 📍 Export Offset Mapping to JSON

The cell below exports the offset mapping to a JSON file for use by other applications.

### 📁 Output File
```
notebooks/test_output/offset_mapping_page3.json
```

### JSON Structure
```json
{
  "pages": [{
    "pageNumber": 3,
    "offset": {"start": 0, "end": 1234},
    "lines": [{
      "content": "1 | Q. Please state your name.",
      "offset": {"start": 0, "end": 30},
      "bbox": "D(3,1.03,1.12,...)",
      "words": [...]
    }]
  }]
}
```

This enables:
- ✅ **Precise page/line citations** - AI agents can cite specific line numbers accurately
- ✅ **Source highlighting** - Use offsets and bboxes to highlight relevant document parts
- ✅ **Traceability** - Map between reflowed content and original OCR positions

In [39]:
from IPython.display import HTML, display

if 'result' in locals() and 'offset_map_3' in locals():
    print("\n🎨 Creating Interactive Offset Visualization...")
    print("="*60)
    
    # Create HTML visualization (renders inline in notebook output)
    viz_html = create_offset_visualization(reflowed_page_3, offset_map_3)
    display(HTML(viz_html))
    
    print("\n📋 Offset Mapping Structure Example:")
    print("="*60)
    
    # Show the structure of offset mapping
    if offset_map_3.pages:
        page = offset_map_3.pages[0]
        print(f"\nPage {page.page_number}:")
        print(f"  offset: {page.offset}")
        
        if page.lines:
            print(f"\n  First line:")
            line = page.lines[0]
            print(f"    content: {line.content[:60]}...")
            print(f"    offset: {line.offset}")
            print(f"    bbox: {line.bbox[:50] if line.bbox else 'N/A'}...")
            
            if line.words:
                print(f"\n    First 3 words:")
                for word in line.words[:3]:
                    start = word.offset['start']
                    end = word.offset['end']
                    print(f"      - '{word.content}' @ [{start}:{end}]")
                    print(f"        bbox: {word.bbox[:40] if word.bbox else 'N/A'}...")
    
    print("\n" + "="*60)
    
    # Export offset mapping as JSON
    print("\n💾 Saving offset mapping to JSON file...")
    
    offset_dict = offset_mapping_to_dict(offset_map_3)
    
    # Save to file
    output_dir = os.path.join(os.getcwd(), 'test_output')
    os.makedirs(output_dir, exist_ok=True)
    
    offset_file = os.path.join(output_dir, 'offset_mapping_page3.json')
    with open(offset_file, 'w') as f:
        json_module.dump(offset_dict, f, indent=2)
    
    print(f"✅ Offset mapping saved to: {offset_file}")
    num_pages = len(offset_dict['pages'])
    print(f"   Pages: {num_pages}")
    total_lines = sum(len(p['lines']) for p in offset_dict['pages'])
    print(f"   Total lines: {total_lines}")
    
else:
    print("⚠️  Required variables not found. Run previous cells first.")


🎨 Creating Interactive Offset Visualization...



📋 Offset Mapping Structure Example:

Page 3:
  offset: {'start': 0, 'end': 291}

  First line:
    content: (B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE O...
    offset: {'start': 0, 'end': 64}
    bbox: D(3,0.9331,0.3227,7.6077,0.3156,7.6079,0.4938,0.93...

    First 3 words:
      - '(B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER' @ [0:64]
        bbox: D(3,0.9331,0.3227,7.6077,0.3156,7.6079,0...


💾 Saving offset mapping to JSON file...
✅ Offset mapping saved to: /workspaces/azure-ai-content-understanding-python/notebooks/test_output/offset_mapping_page3.json
   Pages: 1
   Total lines: 8


## 📄 Reflow Entire Document with Offset Tracking

This cell processes the complete document and saves both the reflowed markdown and the full offset mapping.

### 📁 Output Files
| File | Description |
|------|-------------|
| `notebooks/test_output/legal_transcript_reflowed_with_offsets.md` | Full reflowed document with inline line numbers |
| `notebooks/test_output/offset_mapping_full.json` | Complete offset mapping for all pages |

### In-Memory Variables
| Variable | Type | Description |
|----------|------|-------------|
| `reflowed_full` | `str` | The complete reflowed markdown text |
| `offset_map_full` | `OffsetMapping` | Hierarchical offset data for all pages |

In [40]:
if 'result' in locals():
    print("\n📄 Reflowing entire document with offset tracking...")
    print("="*60)
    
    # Reflow the entire document with offsets
    reflowed_full, offset_map_full = reflow_document_with_offsets(result)
    
    print(f"✅ Document reflowed successfully!")
    print(f"   Total characters: {len(reflowed_full)}")
    print(f"   Total pages: {len(offset_map_full.pages)}")
    
    total_lines = sum(len(p.lines) for p in offset_map_full.pages)
    print(f"   Total lines: {total_lines}")
    
    # Save the reflowed document
    import os
    output_dir = os.path.join(os.getcwd(), 'test_output')
    os.makedirs(output_dir, exist_ok=True)
    
    reflowed_file = os.path.join(output_dir, 'legal_transcript_reflowed_with_offsets.md')
    with open(reflowed_file, 'w', encoding='utf-8') as f:
        f.write(reflowed_full)
    
    print(f"\n💾 Reflowed document saved to: {reflowed_file}")
    
    # Save the complete offset mapping
    offset_dict_full = offset_mapping_to_dict(offset_map_full)
    offset_file_full = os.path.join(output_dir, 'offset_mapping_full.json')
    with open(offset_file_full, 'w') as f:
        json_module.dump(offset_dict_full, f, indent=2)
    
    print(f"💾 Full offset mapping saved to: {offset_file_full}")
    
    # Run accuracy tests on full document
    print("\n🧪 Running accuracy tests on full document...")
    test_results_full = test_offset_accuracy(reflowed_full, offset_map_full)
    for line in test_results_full[:15]:  # Show first 15 lines
        print(line)
    
    print("\n📄 Preview of reflowed content (first 2000 chars):")
    print("="*60)
    print(reflowed_full[:2000])
    print("="*60)
    
    # Create visualization for first page
    print("\n🎨 Creating visualization for first page...")
    viz_html_full = create_offset_visualization(reflowed_full, offset_map_full, highlight_page=1)
    display(HTML(viz_html_full))
    
else:
    print("⚠️  'result' variable not found. Run the analysis cell first.")



📄 Reflowing entire document with offset tracking...
✅ Document reflowed successfully!
   Total characters: 65627
   Total pages: 52
   Total lines: 1390

💾 Reflowed document saved to: /workspaces/azure-ai-content-understanding-python/notebooks/test_output/legal_transcript_reflowed_with_offsets.md
💾 Full offset mapping saved to: /workspaces/azure-ai-content-understanding-python/notebooks/test_output/offset_mapping_full.json

🧪 Running accuracy tests on full document...
🧪 Testing Offset Accuracy...
✅ Line [17:81]: PASS
✅ Line [82:110]: PASS
✅ Line [111:138]: PASS
✅ Line [139:159]: PASS
✅ Line [160:183]: PASS
✅ Line [487:551]: PASS
✅ Line [552:567]: PASS
✅ Line [568:584]: PASS
✅ Line [585:624]: PASS
✅ Line [625:658]: PASS
📊 Results: 10/10 tests passed (100%)

📄 Preview of reflowed content (first 2000 chars):

<!-- Page 1 -->
(B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER
SUPERIOR COURT OF NEW JERSEY
MERCER COUNTY-LAW DIVISION,
DOCKET NO. L-90-2940
IN RE: IN THE MATTER O

In [41]:
# Compare default vs reflowed for page 3
page_number = 3

# Get the original markdown (extract just page 3 content - approximation)
original_lines = markdown.split('\n')

print("📊 COMPARISON: Default vs Reflowed Output")
print("\n" + "=" * 30 + " DEFAULT OUTPUT " + "=" * 30)
print("(Line numbers grouped separately at page bottom)")
print("-" * 76)

# Show a sample of the default output
sample_start = 200
sample_end = 800
print(markdown[sample_start:sample_end])
print("...")

print("\n" + "=" * 30 + " REFLOWED OUTPUT " + "=" * 29)
print("(Line numbers inline with text)")
print("-" * 76)

# Show the reflowed output for page 3
reflowed_page, _ = reflow_document_with_offsets(result, target_page=page_number)
print(reflowed_page[:800])
print("...")

📊 COMPARISON: Default vs Reflowed Output

============================== DEFAULT OUTPUT ==============================
(Line numbers grouped separately at page bottom)
----------------------------------------------------------------------------
ITION OF:

:

Susan Michaud

:

:

Transcript of proceedings taken on July 13, 1990,
at 1 pm, at the office of Mason, Griffin & Pierson, 101 Poor
Farm Road, Princeton, NJ 08540.

682499390

<!-- PageFooter: http://legacy.library.ucsf.e6u/tid/fuq07a00/pdfv.industrydocuments.ucsf.edu/docs/khhl0001 -->
<!-- PageBreak -->


# (B&W) PROTECTED BY MINNESOTA TOBACCO LITIGATION PROTECTIVE ORDER

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25

2

APPEARANCES

On behalf of
Susan Michaud:

MASON, GRIFFIN & PIERSON
BY: Stephanie J. Briody, Esq.
101 Poor Farm Road
Princton, NJ 08540

On b
...

============================== REFLOWED OUTPUT =============================
(Line numbers inline with text)
----------------------------------------

## Using the Standalone Script

For batch processing or command-line usage, you can use the standalone script located at `Customers/Eve Legal/reflow_markdown_with_line_numbers.py`:

```bash
# Process a specific page
python "Customers/Eve Legal/reflow_markdown_with_line_numbers.py" analysis.json --page 3

# Process all pages and save to file
python "Customers/Eve Legal/reflow_markdown_with_line_numbers.py" analysis.json --output reflowed.md

# Custom separator
python "Customers/Eve Legal/reflow_markdown_with_line_numbers.py" analysis.json --separator " | " --output reflowed.md
```

In [42]:
# Example: Run the standalone script on our saved JSON
import subprocess

script_path = os.path.join(os.path.dirname(os.getcwd()), 'Customers', 'Eve Legal', 'reflow_markdown_with_line_numbers.py')
output_file = os.path.join(os.getcwd(), 'test_output', 'legal_transcript_reflowed_script.md')

# Check if the script exists
if not os.path.exists(script_path):
    print(f"⚠️  Standalone script not found at: {script_path}")
    print("   Skipping standalone script test.")
else:
    print(f"🔧 Running standalone reflow script...")
    print(f"   Input: {saved_json_path}")
    print(f"   Output: {output_file}")

    result_code = subprocess.run(
        ['python', script_path, saved_json_path, '--output', output_file],
        capture_output=True,
        text=True
    )

    if result_code.returncode == 0:
        print(f"✅ Script completed successfully!")
        print(result_code.stdout)
    else:
        print(f"❌ Script failed:")
        print(result_code.stderr)

🔧 Running standalone reflow script...
   Input: test_output/legal_transcript_analysis_20260127_155636.json
   Output: /workspaces/azure-ai-content-understanding-python/notebooks/test_output/legal_transcript_reflowed_script.md
✅ Script completed successfully!
Output written to: /workspaces/azure-ai-content-understanding-python/notebooks/test_output/legal_transcript_reflowed_script.md



## Summary

This notebook demonstrated how to:

1. **Extract content** from legal transcripts using Azure Content Understanding's `prebuilt-layout` analyzer
2. **Understand the JSON structure** including the `source` field with bounding box coordinates
3. **Reflow the output** to include line numbers inline with text by:
   - Parsing bounding box coordinates to determine element positions
   - Grouping elements by vertical position (Y coordinate)
   - Matching line numbers with their corresponding text content
4. **Track offsets** for precise citations and source highlighting
5. **Export offset mappings** to JSON for use by other applications

---

### 📁 Output Files Generated

After running all cells, you'll find these files in `notebooks/test_output/`:

| File | Description |
|------|-------------|
| `legal_transcript_reflowed.md` | Reflowed document (markdown only, no offset data) |
| `legal_transcript_reflowed_with_offsets.md` | Full reflowed document with inline line numbers |
| `offset_mapping_page3.json` | Offset mapping for page 3 only |
| `offset_mapping_full.json` | **Complete offset mapping for all pages** |

### 📊 In-Memory Variables

| Variable | Type | Description |
|----------|------|-------------|
| `reflowed_full` | `str` | Complete reflowed markdown |
| `offset_map_full` | `OffsetMapping` | Full offset data (pages → lines → words) |
| `reflowed_page_3` | `str` | Reflowed markdown for page 3 |
| `offset_map_3` | `OffsetMapping` | Offset data for page 3 |

---

### Use Cases

This technique is valuable for:
- **Legal document processing** - Depositions, trial transcripts, court records
- **Academic citations** - Line-numbered source materials
- **Content indexing** - Building searchable databases with line-level citations
- **AI-powered legal research** - RAG applications that need line-accurate references

### Learn More

- [Content Understanding Document Elements](https://learn.microsoft.com/en-us/azure/ai-services/content-understanding/document/elements)
- [Document Markdown Representation](https://learn.microsoft.com/en-us/azure/ai-services/content-understanding/document/markdown)